## Phase 1: Data extraction

https://news.ycombinator.com/item?id=41614126

https://github.com/conjuncts/gmft

https://www.reddit.com/r/LocalLLaMA/comments/1854d06/table_extraction_from_pdf/ ==> some guy did https://docs.google.com/spreadsheets/d/12IhxHZbYF71dPl32PQpF_6pg9e9S8f9W4sTHt-B0KTg/edit?gid=0#gid=0 


https://www.getmagical.com/blog/extract-table-from-pdf


As to whether traditional OCR or LLMs are better for this task

#### Conclusions:
- We REALLY want to keep the right figures, so potentially trying lower end models for this, while it can be accurate a fair amount of the time, likely does not counteract the risk of it eventually hallucinating. Not all LLMs are equal.
- [Marker](https://github.com/vikParuchuri/marker) has 20k stars on github and was found from the excel sheet; but has a commercial usage agreement so Nahh

https://github.com/conjuncts/gmft ? => Relies on Table transformers (microsoft), small, seems ideal. I've also seen stuff for AWS textract but let's keep it simple for now.

In [1]:
import gmft
from gmft.auto import AutoTableDetector, AutoTableFormatter
from gmft.pdf_bindings import PyPDFium2Document
import pandas as pd

# Load PDF and extract tables
pdf_path = "./pdf_inputs/fwc_sample_financial_statement 1.pdf"
detector = AutoTableDetector()
formatter = AutoTableFormatter()

doc = PyPDFium2Document(pdf_path)
tables = []

for page_num, page in enumerate(doc):
    extracted_tables = detector.extract(page)
    for table_index, table in enumerate(extracted_tables):
        formatted_table = formatter.extract(table)
        table_data = formatted_table.df()
        if table_data is not None:
            tables.append(table_data)

doc.close()
print(doc, tables)


<gmft.pdf_bindings.pdfium.PyPDFium2Document object at 0x00000230A3FCB880> [  Motor vehicle expenses Donations Legal costs Administration expenses  \
0                     4H        4G          4F                      4E   
1                 89,000    77,000     296,000               1,649,000   
2                 79,000   120,000      62,000               1,475,000   

  Depreciation Affiliation fees Capitation fees Expenses Employee expenses  \
0           4D               4C              4B                         4A   
1       97,000          228,000         830,000                  3,167,000   
2       88,000          231,000         823,000                  3,023,000   

  revenue      Total Other revenue Rental income Interest  \
0    None       None            3A          None     None   
1    None  7,797,000       613,000       185,000  251,000   
2    None  7,701,000       655,000       244,000  231,000   

  Membership subscriptions          Revenue  \
0                     N

> Tried gmft and it has all these nice metrics etc but when I actually used it, it was terrible - because after some diving in it seems to be designed for actual tables in scientific journals etc, whilst this pdf is amajority of white space, tables go over multiple pages etc so it really does not deal well with this.

Would be way better to just get a text reader for this case
It also takes very long

In [ ]:
import pandas as pd # Sideventure of trying to just throw the pdf at gemini and telling it to generate a csv (DID NOT WORK AT ALL)

# Replace 'your_file.csv' with the actual name of your CSV file
df = pd.read_csv('gemini_test.csv') # wow gemini CANNOT make csvs LOL okay looks like this manual reading off pdf is better atleast.

# Now you can work with the data in the DataFrame 'df'
print(df)

                                                Statement  Item   Category  \
Income                                            Revenue   NaN        NaN   
NaN                              Membership subscriptions   NaN  6748000.0   
NaN                                              Interest   NaN   251000.0   
NaN                                         Rental income   NaN   185000.0   
NaN                                         Other revenue   NaN   613000.0   
...                                                   ...   ...        ...   
NaN                                       Total Cash Used   NaN        NaN   
NaN                    NET CASH FROM FINANCING ACTIVITIES   NaN  8000000.0   
NaN                  NET INCREASE/(DECREASE) IN CASH HELD   NaN  1862000.0   
NaN     Cash and cash equivalents at the beginning of ...   NaN  4955000.0   
NaN     Cash and cash equivalents at the end of the re...   NaN  3093000.0   

        Last Year  Previous Year  
Income        NaN           

In [18]:
import PyPDF2
pdf_path = "fwc_sample_financial_statement 1.pdf"


def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
            text += "\n==================\n"
    return text

pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text)

   
FS 023 Sample financial statement   10 March  2023  | p. 1 Fact Sheet  FS 023 | 10 March  2023  
 
Sam ple financial statements  
 
Statement of  comprehensive income  
For the year ended 30 June  
  
Notes  Last Year  
$ Previous Year  
$ 
Revenue     
Membership subscriptions   6,748,000  6,571,000  
Interest   251,000  231,000  
Rental income   185,000  244,000  
Other revenue  3A 613,000  655,000  
Total revenue   7,797,000  7,701,000  
    
Expenses     
Employee expenses  4A 3,167,000  3,023,000  
Capitation fees  4B 830,000  823,000  
Affiliation fees  4C 228,000  231,000  
Depreciation  4D 97,000  88,000  
Administration expenses  4E 1,649,000  1,475,000  
Legal costs  4F 296,000  62,000  
Donations  4G 77,000  120,000  
Motor vehicle expenses  4H 89,000  79,000  

   
FS 023 Sample financial statement   10 March  2023  | p. 2 Other expenses  4I 835,000  775,000  
Total expenses   7,268,000  6,676,000  
    
Surplus (deficit) for the year   529,000  1,025,000  
    
Other c

In [9]:
import pdfplumber # pdfplumber IS SLOWER BTW by a fair bit, but it also does give much nicer formatting.
pdf_path = "./pdf_inputs/fwc_sample_financial_statement 1.pdf"

def extract_full_text(pdf_path):
    full_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            full_text += page.extract_text() + "\n"  # Append text from each page and add a newline
    return full_text

# Extract all text without caring about page numbers
full_text = extract_full_text(pdf_path)

# Print the entire text
print(full_text)

## Per page version below 
# def extract_pdf_by_pages(pdf_path):
#     pages_text = {}
#     with pdfplumber.open(pdf_path) as pdf:
#         for page_num, page in enumerate(pdf.pages, 1):
#             pages_text[f"page_{page_num}"] = page.extract_text()
#     return pages_text

# # Extract all pages
# pages = extract_pdf_by_pages(pdf_path)

# # Print each page to see what we're working with
# for page_num, text in pages.items():
#     print(page_num, text)

Fact Sheet FS 023 | 10 March 2023
Sample financial statements
Statement of comprehensive income
For the year ended 30 June
Last Year Previous Year
Notes $ $
Revenue
Membership subscriptions 6,748,000 6,571,000
Interest 251,000 231,000
Rental income 185,000 244,000
Other revenue 3A 613,000 655,000
Total revenue 7,797,000 7,701,000
Expenses
Employee expenses 4A 3,167,000 3,023,000
Capitation fees 4B 830,000 823,000
Affiliation fees 4C 228,000 231,000
Depreciation 4D 97,000 88,000
Administration expenses 4E 1,649,000 1,475,000
Legal costs 4F 296,000 62,000
Donations 4G 77,000 120,000
Motor vehicle expenses 4H 89,000 79,000
FS 023 Sample financial statement 10 March 2023 | p. 1
Other expenses 4I 835,000 775,000
Total expenses 7,268,000 6,676,000
Surplus (deficit) for the year 529,000 1,025,000
Other comprehensive income
Gain on revaluation – investment property - 342,000
Total comprehensive income for the year 529,000 1,367,000
The above statement should be read in conjunction with the not

#### Conclusions
PDF plumber takes (noticeably) longer, but is more accurate/has less missing characters so we'll be using this because ideally, the data extraction script should only run once (and we can save the outputs to a DB or something eventually... (foreshadowing: RAG yay!))

## Phase 2 - Data Preprocessing
> Clean useless data out of it (headers etc)

> Segment into useful sections -- the main tables, basically

> Validate data processing

In [10]:
import re
### First, let's just clean the headers/footers since those are absolutely useless (And the Notes $ $ )

def clean_text(raw_text):
    """Removes headers, footers, and page numbers from financial statements."""

    lines = raw_text.splitlines()
    cleaned_lines = []
    for line in lines:
        if not re.search(r"FS 023|Fact Sheet FS 023|p\.\s*\d+", line, re.IGNORECASE): # Improved condition
            cleaned_lines.append(line)

    cleaned_text = "\n".join(cleaned_lines)
    # Get rid of end of document text
    cleaned_text = re.sub(r'© Commonwealth of Australia.*?legal advice\.', '', cleaned_text, flags=re.DOTALL) # DOTALL allows the match to continue across multiple lines
    cleaned_text = re.sub(r'\n+', '\n', cleaned_text).strip()
    return cleaned_text



cleaned_text = clean_text(full_text) 
print(cleaned_text)

Sample financial statements
Statement of comprehensive income
For the year ended 30 June
Last Year Previous Year
Notes $ $
Revenue
Membership subscriptions 6,748,000 6,571,000
Interest 251,000 231,000
Rental income 185,000 244,000
Other revenue 3A 613,000 655,000
Total revenue 7,797,000 7,701,000
Expenses
Employee expenses 4A 3,167,000 3,023,000
Capitation fees 4B 830,000 823,000
Affiliation fees 4C 228,000 231,000
Depreciation 4D 97,000 88,000
Administration expenses 4E 1,649,000 1,475,000
Legal costs 4F 296,000 62,000
Donations 4G 77,000 120,000
Motor vehicle expenses 4H 89,000 79,000
Other expenses 4I 835,000 775,000
Total expenses 7,268,000 6,676,000
Surplus (deficit) for the year 529,000 1,025,000
Other comprehensive income
Gain on revaluation – investment property - 342,000
Total comprehensive income for the year 529,000 1,367,000
The above statement should be read in conjunction with the notes.
Statement of financial position
As at 30 June
Last Year Previous Year
Notes $ $
ASSET

In [11]:
#### So, each main table section ends with "The above statement should be read in conjunction with the notes."

### In this specific case we COULD just do cleaned_text.split('the notes string thing') but I'll use regex just so we can account for a bit more variation in different pdfs.



### If we have more then 4 tables we should just loop through all of them basically...
def split_into_sections_regex(cleaned_text):
    """Splits cleaned financial statement text into 4 sections with flexible matching of the stop phrase."""
    
    # Regex pattern to match variations like:
    # - "The above statement should be read in conjunction with the notes."
    # - "Above statement must be considered along with the notes."
    # - "above statement should be read carefully with the notes."
    stop_pattern = r"(?:The\s*)?above statement .*? with the notes\.?"

    # Split using regex
    sections = re.split(stop_pattern, cleaned_text, flags=re.IGNORECASE)

    # Remove extra whitespace and filter empty sections
    sections = [sec.strip() for sec in sections if sec.strip()]

    if len(sections) != 4:
        print(f"Warning: Expected 4 sections, but found {len(sections)}")

    return sections

# Apply function
segmented_tables = split_into_sections_regex(cleaned_text)

# Assign sections
income_statement_text, balance_sheet_text, equity_changes_text, cash_flow_text = segmented_tables

# Display output
print("\n--- Income Statement ---\n", income_statement_text)
print("\n--- Balance Sheet ---\n", balance_sheet_text)
# print("\n--- Equity Changes ---\n", equity_changes_text)
# print("\n--- Cash Flow ---\n", cash_flow_text)



--- Income Statement ---
 Sample financial statements
Statement of comprehensive income
For the year ended 30 June
Last Year Previous Year
Notes $ $
Revenue
Membership subscriptions 6,748,000 6,571,000
Interest 251,000 231,000
Rental income 185,000 244,000
Other revenue 3A 613,000 655,000
Total revenue 7,797,000 7,701,000
Expenses
Employee expenses 4A 3,167,000 3,023,000
Capitation fees 4B 830,000 823,000
Affiliation fees 4C 228,000 231,000
Depreciation 4D 97,000 88,000
Administration expenses 4E 1,649,000 1,475,000
Legal costs 4F 296,000 62,000
Donations 4G 77,000 120,000
Motor vehicle expenses 4H 89,000 79,000
Other expenses 4I 835,000 775,000
Total expenses 7,268,000 6,676,000
Surplus (deficit) for the year 529,000 1,025,000
Other comprehensive income
Gain on revaluation – investment property - 342,000
Total comprehensive income for the year 529,000 1,367,000

--- Balance Sheet ---
 Statement of financial position
As at 30 June
Last Year Previous Year
Notes $ $
ASSETS
Current Asset

One thing we'll note here is that certain tables (like table 3) are annoying!
Because they have table headers like:

Asset

Revaluation Retained

Reserve Earnings Total

$ $ $

And the way to read this is:
                "Asset revaluation reserve $" "Retained Earnings $" "Total" 
Balance as at ...
...

I wonder if we should go for a generalised function that does any table, or manually do each table here...?
Again, we **COULD** also just try throwing the processed strings into the LLM? Maybe we'll try that

### Manually trying to implement dataframes

Key segments of financial data ARE:

- Income Statement (Revenue, Expenses, Profit)
- Balance Sheet / Financial Position (Assets, Liabilities, Equity)
- Statement of Changes in Equity
- Cash Flow Statement

**We're going to use a way which ASSUMES that pdfs given will follow similarly worded formatting**

In [4]:
segmented_tables[0].split('\n')

['Sample financial statements',
 'Statement of comprehensive income',
 'For the year ended 30 June',
 'Last Year Previous Year',
 'Notes $ $',
 'Revenue',
 'Membership subscriptions 6,748,000 6,571,000',
 'Interest 251,000 231,000',
 'Rental income 185,000 244,000',
 'Other revenue 3A 613,000 655,000',
 'Total revenue 7,797,000 7,701,000',
 'Expenses',
 'Employee expenses 4A 3,167,000 3,023,000',
 'Capitation fees 4B 830,000 823,000',
 'Affiliation fees 4C 228,000 231,000',
 'Depreciation 4D 97,000 88,000',
 'Administration expenses 4E 1,649,000 1,475,000',
 'Legal costs 4F 296,000 62,000',
 'Donations 4G 77,000 120,000',
 'Motor vehicle expenses 4H 89,000 79,000',
 'Other expenses 4I 835,000 775,000',
 'Total expenses 7,268,000 6,676,000',
 'Surplus (deficit) for the year 529,000 1,025,000',
 'Other comprehensive income',
 'Gain on revaluation – investment property - 342,000',
 'Total comprehensive income for the year 529,000 1,367,000']

In [ ]:
import pandas as pd

## THIS ONE IS PRETTY CINEMA NO!?? --> Table 1: Income statements
def extract_income_statement(text):
    """Extracts Revenue, Expenses, and Profit from Income Statement."""
    income_statement = []


    pattern = r"""
        ([A-Za-z\s–]+)        # Item name: letters, spaces, dashes
        \s+                   # One or more spaces
        (?:\d[A-Z])?          # Optional Notes column (e.g., 3A)
        \s*                   # Zero or more spaces
        ([\d,]+|-)            # Last Year value: numbers with commas or a dash
        \s+                   # One or more spaces
        ([\d,]+|-)            # Previous Year value: numbers with commas or a dash
    """
    for line in text.split('\n'):
        # Look for lines with amounts
        # print(line)
        match = re.search(pattern, line, re.VERBOSE)
        if match:
            item = match.group(1).strip()
            last_year = match.group(2).replace(",", "")
            prev_year = match.group(3).replace(",", "")

            # Convert to numeric (handling "-" as zero)
            last_year = 0 if last_year == "-" else float(last_year)
            prev_year = 0 if prev_year == "-" else float(prev_year)

            income_statement.append({"Item": item, "Last Year": last_year, "Previous Year": prev_year})
    
    return pd.DataFrame(income_statement)

def extract_balance_sheet(text):
    """Extracts Assets, Liabilities, and Equity from the Balance Sheet with subsection detection."""
    
    balance_sheet = []
    current_section = None  # Track current mini-table (Assets, Liabilities, etc.)

    pattern = r"""
        ([A-Za-z\s–]+)        # Item name: letters, spaces, dashes
        \s+                   # One or more spaces
        (?:\d[A-Z])?          # Optional Notes column (e.g., 3A)
        \s*                   # Zero or more spaces
        ([\$\d,]+|-)?         # Last Year value: numbers with commas, dollar sign, or dash (optional)
        \s+                   # One or more spaces
        ([\$\d,]+|-)?         # Previous Year value (optional)
    """

    for line in text.split('\n'):
        line = line.strip()

        # If the line is ALL CAPS and has no numbers, treat it as a subsection header
        if line.isupper() and not re.search(r"\d", line):
            current_section = line  # Update section
            continue  # Skip adding this as a data row

        # Match financial data rows
        match = re.search(pattern, line, re.VERBOSE)
        if match:
            item = match.group(1).strip()
            last_year = match.group(2) or "0"  # Default to "0" if empty
            prev_year = match.group(3) or "0"  # Default to "0" if empty

            last_year = last_year.replace(",", "").replace("$", "").strip()
            prev_year = prev_year.replace(",", "").replace("$", "").strip()

            # Convert to numeric (handling "-" as zero, handling empty strings)
            last_year = 0 if last_year in ["-", ""] else float(last_year)
            prev_year = 0 if prev_year in ["-", ""] else float(prev_year)

            balance_sheet.append({
                "Section": current_section,  # Assign to detected subsection
                "Item": item,
                "Last Year": last_year,
                "Previous Year": prev_year
            })

    return pd.DataFrame(balance_sheet)





# Function to apply bold styling where "Total" is in the "Item" column just to make things easier to look at
def bold_style(val):
    f = "font-weight: bold"
    
    # Condition: Check if "Total" appears in the "Item" column
    m = val["Item"].str.contains("Total", case=False, na=False)

    # Create an empty DataFrame of styles
    df1 = pd.DataFrame('', index=val.index, columns=val.columns)

    # Apply bold styling based on condition
    df1.loc[m] = f
    
    return df1


income_statement_df = extract_income_statement(segmented_tables[0])
balance_sheet = extract_balance_sheet(segmented_tables[1])

print("\nIncome Statement:")
# display(income_statement_df.style.apply(bold_style, axis=None)) # Just bolds any column that has "Total" in the item section
display(income_statement_df)
display(balance_sheet)
display(extract_income_statement(segmented_tables[2]))



"""
The issue I'm coming across with right now is that: Well, there's too much variation.

- The tables aren't the same; some subtables within the main sections go with full caps, some don't (Table 1 vs 2) Expenses vs LIABILITIES
And later on in Table 3 you have multiline headers (Asset Revaluation Reserve) which messes with the regex of the previous commands.

This is technically doable, but overall this means you'd have to determine what table type it is first if you want a properly 1:1 accurate table.

For approximations you could get away with this, and maybe create a generalised table that would work alright.

However, consider when we're scaling this up -- our regex assumes 3 column tables for instance.
There may be other bugs and different kinds of multilined strings, or simply just malformatted docs. What then? This would require significant time and effort to come to solutions to, using a manual method.

"""


Income Statement:


,Item,Last Year,Previous Year
0,Membership subscriptions,6748000.0,6571000.0
1,Interest,251000.0,231000.0
2,Rental income,185000.0,244000.0
3,Other revenue,613000.0,655000.0
4,Total revenue,7797000.0,7701000.0
5,Employee expenses,3167000.0,3023000.0
6,Capitation fees,830000.0,823000.0
7,Affiliation fees,228000.0,231000.0
8,Depreciation,97000.0,88000.0
9,Administration expenses,1649000.0,1475000.0


,Section,Item,Last Year,Previous Year
0,None,As at,30.0,0.0
1,None,Notes,0.0,0.0
2,ASSETS,Cash and cash equivalents,3093000.0,4955000.0
3,ASSETS,Trade and other receivables,528000.0,226000.0
4,ASSETS,Other current assets,408000.0,383000.0
5,ASSETS,Total Current Assets,4029000.0,5564000.0
6,ASSETS,Land and buildings,10000000.0,0.0
7,ASSETS,Plant and equipment,104000.0,132000.0
8,ASSETS,Motor vehicles,141000.0,147000.0
9,ASSETS,Investment property,2170000.0,2000000.0


,Item,Last Year,Previous Year
0,July previous year,0.0,6186000.0
1,Revaluation increment,342000.0,0.0
2,Profit for the year,0.0,1367000.0
3,Closing balance as at,342000.0,6576000.0
4,Revaluation increment,0.0,0.0
5,Profit for the year,0.0,529000.0
6,Closing balance as at,342000.0,7105000.0


"\nThe issue I'm coming across with right now is that: Well, there's too much variation.\n\n- The tables aren't the same; some subtables within the main sections go with full caps, some don't (Table 1 vs 2) Expenses vs LIABILITIES\nAnd later on in Table 3 you have multiline headers (Asset Revaluation Reserve) which messes with the regex of the previous commands.\n\nThis is technically doable, but overall this means you'd have to determine what table type it is first if you want a properly 1:1 accurate table.\n\nFor approximations you could get away with this, and maybe create a generalised table that would work alright.\n"

Anyways I think I've spent way too much time on this section because I impliclty thought we *needed* to get it to a pandas dataframe, and spent time figuring out hwo to deal with multiline field names, etc...

However, can't we just throw the tables (from the clean segmentation process) into a generative model?

# Phase 3 - GENAI Models Yay!!!
Leverage Existing Models:
You might choose to fine-tune or prompt an existing language model (e.g., GPT-4) to further interpret these segments. The goal is to have the model:

- Identify and extract key metrics from each section.
- Understand the context and handle long documents efficiently.


Prompt Engineering: Design prompts that guide the model to extract numbers accurately.



####    I SEE THE PRIVACY ISSUES NOT TO WORRY. -- Sending data to OPENAI etc... Local models!?!?
So, in the case of private information looking into local models would definitely be an idea, but... I have 4GB vram on my PC.


I would try to use local models but unfortunately my PC has spec limitations so :(
    
    ... IS WHAT I WOULD SAY IF I WASN'T TAKING THIS SERIOUSLY! Jkjk, but seriously my GPU has 4GB vram which is horrendous, so I can try to run very small models but I don't think they're that trustworthy -- I'll do it anyways but yeah..



In [6]:
from ollama import chat, generate
MODEL = "deepseek-r1:latest"

TEMP = 0 

# stream = chat(
#     model=MODEL,
#     messages=[{"role": "user", "content": "What is Machine Learning"}],
#     options={"temperature": TEMP},
#     stream=True,
# )

# for chunk in stream:
#     print(chunk["message"]["content"], end="", flush=True)

### Example 
def prompt_financial_extraction(section_text):
    """Uses Ollama's Python API to extract financial metrics from a text section."""
    response = generate(
        model=MODEL,  # Your model
        prompt=(
            "Extract the following financial metrics from the text: "
            "Revenue, Net Income, Operating Expenses, and Cash Flow. "
            "Return the data in JSON format. Text:\n" + section_text
        ),
        format="json"  # Ensures JSON output
    )

    return response["response"]  # Extract model response

# Example usage
section_text = "Total revenue was $7,797,000, with expenses totaling $7,268,000. Net income was $529,000."

json_data = prompt_financial_extraction(section_text)

# Print extracted financial metrics
print(json_data)

{ "Revenue": "$7,797,000", "Net Income": "$529,000", "Operating Expenses": "$7,268,000" }



 
 




 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 




In [ ]:
from ollama import chat, generate
import json
MODEL = "deepseek-r1:1.5b" ## Using the SMALLEST version of deepseek because my GPU etc is so *@.! that the 7b takes too long to properly work.

## IDEALLY I'D LIKE TO USE any 70B FOR ANY TASKS LIKE THIS -- I feel like 70b is personally the minimum threshold I would use if I was developing anything and needed specific tasks like this to happen -- otherwise, it can literally go rogue and spiral into some nonsense just because of its small parameter count.
# 
# During experimenting, I mispelt a single word with the 7b model in the chat feature and it started speaking spanish. I'm not joking. 

## Like the 1.5b model is JUST SILLY

TEMP = 0 

def prompt_financial_extraction(section_text):
    """Uses Ollama's Python API to extract financial metrics from a text section."""
    response = generate(
        model=MODEL,  # Your model
        prompt=(
            "Extract financial information from the following text and return it in a markdown table." + section_text
        ),
    )
    return response 
# Example usage
section_text = "Total revenue was $7,797,000, with expenses totaling $7,268,000. Net income was $529,000."
json_data = prompt_financial_extraction(section_text)
print(json_data)
print(json_data.response)

model='deepseek-r1:1.5b' created_at='2025-02-11T12:11:30.5496727Z' done=True done_reason='stop' total_duration=5736155000 load_duration=29082400 prompt_eval_count=57 prompt_eval_duration=8000000 eval_count=383 eval_duration=5697000000 response='<think>\nOkay, so I need to extract financial information from the given text and put it into a markdown table. Let me see what\'s in there.\n\nThe user provided this text: "Total revenue was $7,797,000, with expenses totaling $7,268,000. Net income was $529,000."\n\nFirst, I should identify the key financial figures here. There\'s total revenue, which is $7,797,000. Then there are expenses, $7,268,000. After that, net income, which is $529,000.\n\nI\'ll need to present this in a table with columns like Total Revenue, Expenses, and Net Income. So the structure would be:\n\n| Item      | Amount   |\n|-----------|---------|\n| Total Revenue | $7,797,000 |\n| Expenses  | $7,268,000 |\n| Net Income | $529,000 |\n\nWait, does that cover all the point

In [35]:
json_data = prompt_financial_extraction(segmented_tables[0])
print(json_data.response)


<think>
Okay, I need to extract financial information from the provided text and create a markdown table. Let me go through each section step by step.

First, I'll start with Statement of Comprehensive Income because that's one of the main sections. The text mentions total comprehensive income for both years, so I can note those as 529,000 for last year and 1,367,000 for previous year.

Next up is Statement of Changes in Equity. This section includes equity at end of year and changes during the year. The ending equity was $48,298,000 last year and $51,769,000 previously. Also, there's a net increase of 384,000 over the years.

Then, Income from Operations is split into revenue and expenses. Revenue last year was $7,797,000 and before was $7,701,000. Expenses include employee costs at $3,167,000 down from $3,023,000, capital expense $4B at $830,000 from $823,000, and others totaling $1,295,000. The net income for last year was $156,000 versus $167,000 before.

Capital Expenditure is $44

Here's  a table I got from 1.5b somehow... I think it got a bit lucky, but again you can see multiple occurrences of the same values, etc..
Notes in last year columns...
| Financial Metric              | Last Year | Previous Year |
|-------------------------------|------------|--------------|
| **Statement of comprehensive income** | Revenue                 | Revenue             |
| Total revenue                | $7,797,000     | $7,701,000       |
| Membership subscriptions         | 6,748,000      | 6,571,000        |
| Interest                    | 251,000       | 231,000          |
| Rental income               | 185,000       | 244,000          |
| Other revenue                | 3A                  | 613,000          |
| Total revenue                | $7,797,000     | $7,701,000       |
| Expenses                    |Employee expenses    |Employee expenses    |
| Total employee expenses        | 4A                   | 3,167,000      |
| Capitation fees               | 4B                  | 830,000          |
| Affiliation fees            | 4C                  | 228,000          |
| Depreciation                 | 4D                   | 97,000           |
| Administration expenses     | 4E                  | 1,649,000      |
| Legal costs                | 4F                  | 296,000          |
| Donations                    | 4G                  | 77,000           |
| Motor vehicle expenses    | 4H                   | 89,000           |
| Total expenses             | $7,268,000     | $6,676,000      |
| Surplus (deficit)            | 529,000         | 1,025,000       |
| **Other comprehensive income**   | Gain on revaluation – investment property - | Total comprehensive income for the year |
| Other comprehensive income     | 342,000          | 775,000           |
| Other comprehensive income     | Total comprehensive income for the year |
| 529,000                  | $1,367,000       |

In [2]:
#### Try openai gpt4 API - I just tried it on gemini and it one shotted it easy... https://aistudio.google.com/prompts/1MR2KJksnFwgQ9kXEUsDwnm4rdLnUh9WX. System prompt: "Extract financial information from the following text and return it in a markdown table."
# Gemini has a free API AND an insane context window of 1 million to 2 million tokens depending on the model so we'll use that.

from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("GEMINI_KEY")
if not api_key:
    raise ValueError("No API key found. Please check your .env file.")
print("API key loaded via .env!")

API key loaded via .env!


In [ ]:
# from IPython.display import display, Markdown, Latex
# display(Markdown(response.text)) 

#### This is just proof that gemini can easily split up a primary table (such as the income statement) into the subsections easily if you so want to. For no

| Revenue                      | Last Year ($) | Previous Year ($) |
| ---------------------------- | ------------- | ----------------- |
| Membership subscriptions     | 6,748,000     | 6,571,000         |
| Interest                     | 251,000       | 231,000           |
| Rental income                | 185,000       | 244,000           |
| Other revenue                | 613,000       | 655,000           |
| **Total revenue**            | **7,797,000** | **7,701,000**     |

| Expenses                   | Last Year ($) | Previous Year ($) |
| -------------------------- | ------------- | ----------------- |
| Employee expenses          | 3,167,000     | 3,023,000         |
| Capitation fees            | 830,000       | 823,000           |
| Affiliation fees           | 228,000       | 231,000           |
| Depreciation               | 97,000        | 88,000            |
| Administration expenses    | 1,649,000     | 1,475,000         |
| Legal costs                | 296,000       | 62,000            |
| Donations                  | 77,000        | 120,000           |
| Motor vehicle expenses     | 89,000        | 79,000            |
| Other expenses             | 835,000       | 775,000           |
| **Total expenses**         | **7,268,000** | **6,676,000**     |

| Income Summary                      | Last Year ($) | Previous Year ($) |
| ----------------------------------- | ------------- | ----------------- |
| Surplus (deficit) for the year      | 529,000       | 1,025,000         |
| Gain on revaluation – investment property | -           | 342,000           |
| **Total comprehensive income for the year** | **529,000** | **1,367,000**     |


In [1]:
## Yeah.... GEMINI just solves this lmao.. code up tomorrow.
from google import genai
import google.genai.types as types

## Using free tier so 15RPM w/ 1 million context window


def gemini_financial_extraction(tabular_text: str, model: str = "gemini-2.0-flash"):
    """Extracts financial data from text and returns it as CSV-formatted text."""
    client = genai.Client(api_key=api_key)
    
    response =  client.models.generate_content(
    model="gemini-2.0-flash", 
    config=types.GenerateContentConfig(
        system_instruction="Extract financial information from the following text and return it in a JSON FORMAT with two primary keys:\n 1. `table_name`: (The name of the financial table (e.g 'Income Statement')).\n 2. `csv_data`:The extracted financial data in valid CSV format. .\n Ignore notes if they exist and replace any '-' sections with a 0. DO NOT ADD OR REMOVE NEGATIVE SIGNS THAT DO NOT EXIST.",
        ),
    contents=tabular_text
    )

    return response
response = gemini_financial_extraction(segmented_tables[0])
print(response.text)





NameError: name 'segmented_tables' is not defined

In [ ]:
import os
import json

# Function to extract JSON and remove code block formatting
def clean_json_response(text):
    """Extracts valid JSON from text by removing markdown code blocks and extra text."""
    # Remove ```json ... ``` wrapping
    text = re.sub(r"```json\s*", "", text)  # Remove opening block
    text = re.sub(r"\s*```$", "", text)  # Remove closing block
    
    # Extract only the valid JSON part
    match = re.search(r"\{.*\}", text, re.DOTALL)  # Match JSON structure
    if match:
        return match.group(0)  # Return extracted JSON
    return None

# Loop through all segmented tables and process them
for idx, table_text in enumerate(segmented_tables):
    json_data = gemini_financial_extraction(table_text).text  # Get JSON response
    # print(json_data)

    json_data = clean_json_response(json_data)
    print(json_data)
    try:
        parsed_data = json.loads(json_data)  # Convert text response to Python dictionary
        
        table_name = parsed_data.get("table_name", f"financial_table_{idx+1}")  # Default name if missing
        csv_content = parsed_data.get("csv_data", "")  # Extract CSV content

        # Ensure a valid filename
        file_name = f"./data/{table_name.replace(' ', '_').lower()}.csv"

        # Save the CSV data to a file
        with open(file_name, "w", encoding="utf-8") as file:
            file.write(csv_content)

        print(f"✅ Saved: {file_name}")

    except json.JSONDecodeError:
        print(f"❌ Failed to parse JSON for table {idx+1}")
        break

```json
{
  "table_name": "Statement of comprehensive income",
  "csv_data": " ,Last Year,Previous Year\nNotes,$,$\nRevenue,,\nMembership subscriptions,6748000,6571000\nInterest,251000,231000\nRental income,185000,244000\nOther revenue,613000,655000\nTotal revenue,7797000,7701000\nExpenses,,\nEmployee expenses,3167000,3023000\nCapitation fees,830000,823000\nAffiliation fees,228000,231000\nDepreciation,97000,88000\nAdministration expenses,1649000,1475000\nLegal costs,296000,62000\nDonations,77000,120000\nMotor vehicle expenses,89000,79000\nOther expenses,835000,775000\nTotal expenses,7268000,6676000\nSurplus (deficit) for the year,529000,1025000\nOther comprehensive income,,\nGain on revaluation – investment property,0,342000\nTotal comprehensive income for the year,529000,1367000"
}
```
{
  "table_name": "Statement of comprehensive income",
  "csv_data": " ,Last Year,Previous Year\nNotes,$,$\nRevenue,,\nMembership subscriptions,6748000,6571000\nInterest,251000,231000\nRental income,185

### Notes:

So, it even knows that (brackets) signify negatives/losses due to the context of financial statements, which is something sneaky that I have suspicions
even a 70b local model would not capture properly (so you'd have to either tell it in the prompt, or for less uncertainty and ideally, convert brackets into negatives in the preprocessing section).

#### On Using APIs in this case
Of course privacy is a concern but in this case, financial statements are often publicly disclosed documents and this specific instance is a sample assessment so it's fine.

If we had to use private documents, I would definitely recommend looking into setting up a 70b local model (not expensive at all to do btw, any small business with some money can get a strong enough gpu for this).



## Task 4 - Generate summary report
(Using AI)

> Key fianncial metrics, notable trends or observations
>  Short nrrative summary in natural language

In [3]:
import glob

def read_csv_files(directory="./data"):
    """Reads all CSV files in a directory and returns their content as a single string."""
    csv_files = glob.glob(f"{directory}/*.csv")  # Get all CSV files in the directory
    all_csv_text = ""  # Initialize an empty string

    for file in csv_files:
        with open(file, "r", encoding="utf-8") as f:
            all_csv_text += f"\n### {file.split('/')[-1].replace('.csv', '').title()} ###\n"
            all_csv_text += f.read()  # Append file content
            all_csv_text += "\n"

    return all_csv_text

# Example usage
csv_text = read_csv_files("./data")  # Reads all CSVs as a string
# print(csv_text)


In [6]:
        # system_instruction="Generate a summary report in markdown that highlights the financial health of the company, given the following tables. The report should include:"
        # "Key financial metrics (revenue, net income, etc.)"
        # "Any notable trends or observations"
        # "A short narrative summary in natural language"
        # ),
def generate_summary_report(tables: str):
    """Based on the information of the segmented tables for a set of csvs, generate a summary report highlighting the financial health of the company."""

    client = genai.Client(api_key=api_key)
    
    response =  client.models.generate_content(
    model="gemini-2.0-flash", 
    config=types.GenerateContentConfig(
        system_instruction="Generate a summary report in markdown that highlights the financial health of the company, given the following tables. The report should include:"
        "Key financial metrics (revenue, net income, etc.). You can display this in markdown tables, with an extra column for notes."
        "Any notable trends or observations"
        "A short narrative summary in natural language at the end."
        ),
    contents=tables,
    )

    print(f"Used {response.usage_metadata.total_token_count} tokens in total to generate summary report.")

    return response
response = generate_summary_report(csv_text)

print(response.text)
from IPython.display import display, Markdown
display(Markdown(response.text))

    

Used 2156 tokens in total to generate summary report.
## Financial Health Report

Here's a summary of the company's financial health based on the provided data.

### Key Financial Metrics

| Metric                     | Last Year $ | Previous Year $ | Notes                                                                         |
| -------------------------- | ----------- | ------------- | ----------------------------------------------------------------------------- |
| **Total Revenue**          | 7,797,000   | 7,701,000     | Slight increase from previous year, driven primarily by membership subscriptions.   |
| **Net Income**             | 529,000     | 1,025,000     | Significant decrease compared to the previous year.                             |
| **Cash from Operations**   | 536,000     | 902,000       | Decrease in cash generated from operations.                                    |
| **Cash at End of Period**  | 3,093,000   | 4,955,000     | Significant decrease in cash holdi

## Financial Health Report

Here's a summary of the company's financial health based on the provided data.

### Key Financial Metrics

| Metric                     | Last Year $ | Previous Year $ | Notes                                                                         |
| -------------------------- | ----------- | ------------- | ----------------------------------------------------------------------------- |
| **Total Revenue**          | 7,797,000   | 7,701,000     | Slight increase from previous year, driven primarily by membership subscriptions.   |
| **Net Income**             | 529,000     | 1,025,000     | Significant decrease compared to the previous year.                             |
| **Cash from Operations**   | 536,000     | 902,000       | Decrease in cash generated from operations.                                    |
| **Cash at End of Period**  | 3,093,000   | 4,955,000     | Significant decrease in cash holdings.                                        |
| **Total Assets**           | 16,444,000  | 7,843,000     | Substantial increase in total assets, primarily due to the purchase of land and buildings.  |
| **Total Liabilities**      | 8,997,000   | 925,000       | Significant increase in total liabilities, primarily due to borrowings.         |
| **Retained Earnings**      | 7,105,000   | 6,576,000     | Increase in retained earnings, reflecting profitability over time.             |

### Trends and Observations

*   **Revenue Stability:** Revenue remained relatively stable year-over-year, with a marginal increase. Membership subscriptions are a key driver.
*   **Decreased Profitability:** Net income significantly decreased compared to the previous year.
*   **Major Investment:** A large purchase of land and buildings significantly impacted the company's asset base and cash flow.
*   **Increased Borrowing:** The company took on a substantial amount of debt to finance its investment activities.
*   **Decreased Cash Position:** Cash holdings decreased significantly due to investment activities.

### Summary

The company experienced a year of significant investment, primarily in land and buildings, leading to substantial growth in total assets. This investment was financed through borrowings, resulting in a significant increase in liabilities and a decrease in cash reserves. While revenue remained stable, net income decreased compared to the previous year. The key takeaway is that the company made a strategic investment that affected cash flow in the short term and will likely change the strategic direction of the company, increasing assets with the land and building purchase.


In [7]:
from markdown_pdf import MarkdownPdf, Section

def save_markdown_to_pdf(markdown_text, output_path="./reports/summary_report.pdf", debug=False):
    """Converts Markdown text to a PDF with proper formatting and saves it."""

    # Ensure the markdown starts with a Level 1 header (H1) to prevent TOC errors
    if not markdown_text.strip().startswith("# "):
        markdown_text = "# Financial Summary Report\n\n" + markdown_text  # Add default title


    # If debug mode is on, check if the file exists and create a unique filename
    if debug and os.path.exists(output_path):
        base, ext = os.path.splitext(output_path)  # Split filename and extension
        counter = 1

        # Keep incrementing filename until we find a free one
        while os.path.exists(f"{base}{counter}{ext}"):
            counter += 1

        output_path = f"{base}{counter}{ext}"


    pdf = MarkdownPdf(toc_level=3)  # Include headings up to level 3 in TOC

    # Add the markdown content as a section
    pdf.add_section(Section(markdown_text))

    # Save the PDF
    pdf.save(output_path)

    print(f"✅ Summary saved as {output_path}")

# Example usage
summary_markdown = response.text  # Get markdown response from LLM
save_markdown_to_pdf(summary_markdown, debug=True)


✅ Summary saved as ./reports/summary_report4.pdf


Summary reports 2-4 use the same prompt, I wanted to see how much variance there is.

There exists some variance in the key financial metrics given, but overall it still tells you all the main notable trends and observations, as well as a good summary of the business situation (made a big play w.r.t buying land, so they should try and play safe/conservatively now to reap these rewards later since they currently have a LOT of liabilities.)

## Phase 5 - Scaling And Stuff

#### For scaling preprocessing

We currently use a mix of manutal table names w/ regex patterns which is fine here, but in more varied datasets we may have some errors as people use slightly different words/terminology that falls through the regex patterns. So potentially using NLP based methods (like spacy, etc.) would help here. 

(Ideally, the pdfs we'd have aren't spaced messes and actually have concise tables so we can use gmft but ideals do not always happen).


#### For RAG:

Potential for RAG for greater reliability or some sort of pre-processed database so the PDF reading and processing doesn't have to happen every time.

Then you RAG it to get more accurate answers while still being able to ask the GenAI model any query.

RAG is a trade-off of speed for accuracy, but for large databases, it’s pretty practical because of context limits (i.e., you can’t throw 10,000 docs into the LLM, so use a retriever to find 5 relevant documents instead!).

// Optimise retrieval via pre-indexing statements

// Deploying on cloud (AWS lambda) for better pipelines and scalability (e.g preprocessing function runs whenever new pdf is inputted, then thrown into a 
database)



//// https://gemini.google.com/app/92f6d574bc081ed7 ====> for other scaling ideas like docker etc